In [1]:
"""
    Copyright (c) 2024 Idiap Research Institute, http://www.idiap.ch/
    Written by Cem Bilaloglu <cem.bilaloglu@idiap.ch>

    This file is part of tactileErgodicExploration.

    tactileErgodicExploration is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License version 3 as
    published by the Free Software Foundation.

    tactileErgodicExploration is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with tactileErgodicExploration. If not, see <http://www.gnu.org/licenses/>.
"""

import numpy as np

np.set_printoptions(formatter={"float": lambda x: "{0:0.3e}".format(x)})

import scipy.sparse.linalg as sla

import robust_laplacian
import time


from plotting_utils import *
from pointcloud_utils import *
from virtual_agents import FirstOrderAgent, SecondOrderAgent
import plotly.graph_objects as go

point_cloud_dir = "point_clouds/"
data_dir = "data/"  # for saving the experiment data

### Inspired from the work 

Heat Equation Driven Area Coverage (HEDAC):

Ivić, S., Crnković, B., & Mezić, I. (2017). Ergodicity-Based Cooperative Multiagent Area Coverage via a Potential Field. IEEE Transactions on Cybernetics, 47(8), 1983–1993. https://doi.org/10.1109/TCYB.2016.2634400

Spectral acceleration using eigenbasis:

Sharp, N., Attaiki, S., Crane, K., & Ovsjanikov, M. (2022). DiffusionNet: Discretization Agnostic Learning on Surfaces. ACM Transactions on Graphics, 41(3), 27:1-27:16. https://doi.org/10.1145/3507905

Heat method: 

Crane, K., Weischedel, C., & Wardetzky, M. (2013). Geodesics in heat: A new approach to computing distance based on heat flow. ACM Transactions on Graphics, 32(5), 152:1-152:11. https://doi.org/10.1145/2516971.2516977

Discrete Laplacian of the point cloud:

Sharp, N., & Crane, K. (2020). A Laplacian for Nonmanifold Triangle Meshes. 
Computer Graphics Forum, 39(5), 69–80. https://doi.org/10.1111/cgf.14069

In [2]:
# Select the object to explore

# obj_name = "bun270_X" # Stanford bunny with X projected as the target
obj_name = "plate_shapes"  # random IKEA plate with hand-drawn shapes
# obj_name = "cup_X" # random cup that we scanned with X projected as the target

experiment_index = 2  # choose which initial position to use from x0_arr_10.npz

class param:
    pass  # c-style struct


param.timesteps = 500  # total simulation timesteps

# tuning: [1,100] increasing alpha result in global exploration closer to SS
# decreasing alpha result in local exploration lower limited
param.alpha = 100
# tuning: [25, 300] increasing the value bring more flexibility in selecting
# the pairing alpha. Lowering the value results in faster computation
param.nb_eigen = 200

# eigen corresponds to using Laplacian eigenbasis and using spectral acceleration
# when integrating the diffusion (heat) equation
# exact corresponds to using implicit time stepping for integrating
param.method = "eigen"
# param.method = "exact"

# voxel filter size for downsampling the point cloud
param.voxel_size = 0.003
# radius for the agent footprint that'd be used in coverage
param.agent_radius = 2.5 * param.voxel_size # for the cup and the bunny
# param.agent_radius = 5 * param.voxel_size  # for the plate
# define speed and acceleration in terms of voxel size
param.max_velocity = 1 * param.voxel_size
param.max_acceleration = 1 * param.max_velocity

# tuning: doesn't have much effect on exploration so we keep it at 1
param.source_strength = 1

# number of iterations for the non-stationary heat equation
# we kept it at 1 in this work to decrease the computational cost and
# it was enough for the exploration task
param.ndt = 1
# max. num. of neighbors to consider for computing the neighbors in agent radius
param.nb_max_neighbors = 500
# num. of neighbors to consider for tangent space and gradient computation
param.nb_minimum_neighbors = 20
# num. of neighbors to consider for implicitly determining the boundary
# setting this lower in bunny resutls in right ear considered as a seperate body
# setting this higher in bunny results in the right ear being considered as part
# of the main body
param.nb_boundary_neighbors = 40

In [3]:
# Select the object and load the point cloud
# ==========================================
filename = f"{point_cloud_dir}{obj_name}.ply"
pcloud = process_point_cloud(filename, param)

Original Point cloud with 7876 points is downsampled with voxel size 0.003
 resulted in 6228 points
dt: 6.668e-04, h: 2.582e-03, s: 3.000e-03


In [4]:
# Get the discrete Laplacian on the point cloud
# =============================================
"""
Sharp, N., & Crane, K. (2020). A Laplacian for Nonmanifold Triangle Meshes. 
Computer Graphics Forum, 39(5), 69–80. https://doi.org/10.1111/cgf.14069
compute the Laplacian of the point cloud
"""
start_time = time.time()
n = 1  # number of iterations !1 only if we want to measure the time complexity
for _ in range(n):  # only for measuring the time complexity
    C, M = robust_laplacian.point_cloud_laplacian(
        pcloud.vertices, n_neighbors=param.nb_boundary_neighbors
    )
print(f"Process finished --- {((time.time() - start_time)/n):.3e} seconds ---")

Process finished --- 1.163e-01 seconds ---


It is possible to use other methods for solving the heat equation, but here we use Laplacian eigengunctions.

$$L\phi_i=\lambda_iM\phi_i$$

$\lambda_i$ corresponding to first $k$ smallest eigenvalues and $\phi_i$ is the i'th eigenvector , $M$ is the mass matrix and $L$ is weak laplacian then heat equation approximation becomes @sharpDiffusionNetDiscretizationAgnostic2022

$$h_t(u)=\Phi \left[
    {\begin{array}{c}
        e^{-\lambda_1 t}\\
        e^{-\lambda_2 t}\\
		\vdots
    \end{array}}
\right]
\odot (\Phi^TMu)
$$

right side of Hadamard Product is spectral weights, left side of Hadamard Product is projecting back to vertices


In [5]:
# Preprocessing step for solving diffusion
# only needs to be done once the point cloud changes
# ================================================================
start_time = time.time()
n = 1  # number of iterations !1 only if we want to measure the time complexity
for _ in range(n):  # only for measuring the time complexity
    if param.method == "exact":
        A_inv = sla.inv(M + pcloud.dt * C)
        A_invM = A_inv @ M
    elif param.method == "eigen":
        # Laplacian eigenfunctions
        # compute the eigenvalue decomposition of Laplace-Beltrami
        evals, evecs = sla.eigsh(C, param.nb_eigen, M, sigma=1e-12)
        Phi = evecs
        exp_vector = np.zeros(param.nb_eigen)
        for i in range(param.nb_eigen):
            exp_vector[i] = np.exp(-evals[i] * pcloud.dt)
        first_term = Phi @ exp_vector
        PhiT_M = Phi.T @ M
        print(f"First 5 eigenvalues: {(evals[:5])}")
print(f"Process finished in --- {((time.time() - start_time)/n):.3e} seconds ---")

First 5 eigenvalues: [-5.774e-13 2.205e+02 2.287e+02 6.219e+02 6.290e+02]
Process finished in --- 2.472e+00 seconds ---


In [6]:
# Integration step for solving diffusion
# called at each timestep during the runtime
# ================================================================
"""
Spectral acceleration using eigenbasis:
Sharp, N., Attaiki, S., Crane, K., & Ovsjanikov, M. (2022). 
DiffusionNet: Discretization Agnostic Learning on Surfaces. 
ACM Transactions on Graphics, 41(3), 27:1-27:16. https://doi.org/10.1145/3507905
"""
start_time = time.time()
n = 1  # number of iterations !1 only if we want to measure the time complexity
for _ in range(n):  # only for measuring the time complexity
    if param.method == "exact":
        ut = A_invM @ pcloud.u0
    elif param.method == "eigen":
        ut = pcloud.u0
        second_term = PhiT_M @ ut
        third_term = exp_vector * second_term
        ut = Phi @ third_term
print(f"Process finished --- {(time.time() - start_time)/n} seconds ---")

Process finished --- 0.006429910659790039 seconds ---


In [7]:
#  Initialize the agent and visualize the initial position and the point cloud
# ===============================
nb_experiments = 10

x0_arr = np.load(f"{data_dir}{obj_name}/x0_arr_{nb_experiments}.npz")["x0_arr"]
# agent = FirstOrderAgent(
#     x=np.zeros(3), dim_t=param.timesteps, max_velocity=param.max_velocity
# )

agent = SecondOrderAgent(
    x=np.zeros(3), dim_t=param.timesteps, max_velocity=param.max_velocity,max_acceleration=param.max_acceleration*2
)
agent.x = x0_arr[experiment_index - 1, :]
agent.radius = param.agent_radius

plots = []
# visualize the agents' initial position
x = agent.x
scatter = go.Scatter3d(
    x=[x[0]],
    y=[x[1]],
    z=[x[2]],
    name="Agent",
    marker=dict(
        size=8,
        color="rgb(0,255,0)",
    ),
    line=dict(width=0.1),
)
plots.append(scatter)
visualize_point_cloud(pcloud.vertices, colors=pcloud.u0, plots=plots, is_show_plot=True)

FigureWidget({
    'data': [{'line': {'width': 0.1},
              'marker': {'color': 'rgb(0,255,0)', 'size': 8},
              'name': 'Agent',
              'type': 'scatter3d',
              'uid': 'e4342c4b-15ca-4f46-91a9-81b8013aaf85',
              'x': [0.5957466661930084],
              'y': [0.3105558007955551],
              'z': [0.08647464960813522]},
             {'marker': {'color': array([0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00, 2.550e+02]),
                         'colorscale': [[0.0, 'rgb(0,0,255)'], [1.0,
                                        'rgb(255,0,0)']],
                         'opacity': 1,
                         'size': 1},
              'mode': 'markers',
              'name': 'Point Cloud',
              'type': 'scatter3d',
              'uid': '46916c79-6b4d-4bb8-af4e-aa0dba862a45',
              'x': array([5.143e-01, 5.307e-01, 5.237e-01, ..., 4.702e-01, 5.210e-01, 4.641e-01]),
              'y': array([4.150e-01, 4.125e-01, 4.1

In [8]:
def hedac(agent, param, pcloud):
    """
    Perform HEDAC exploration using the agent and the point cloud.

    Original implementation on a 2-D rectangular grid by Ivic et al.
    Ivić, S., Crnković, B., & Mezić, I. (2017). Ergodicity-Based Cooperative 
    Multiagent Area Coverage via a Potential Field. IEEE Transactions on 
    Cybernetics, 47(8), 1983–1993. https://doi.org/10.1109/TCYB.2016.2634400

    Args:
        agent (Agent): The agent object representing the virtual exploration agent.
        param (Parameters): The parameters for the HEDAC algorithm.
        pcloud (PointCloud): The point cloud object representing the exploration
             domain and target

    Returns:
        tuple: A tuple containing the agent's trajectory, heat array,
            coverage array, and time array.
    """
    coverage_arr = np.zeros((len(pcloud.vertices), param.timesteps))
    heat_arr = np.zeros_like(coverage_arr)

    # we normalize the goal because it should be a probability distribution
    goal_density = normalize_mat(pcloud.u0)

    # we keep this and add coverage at each timestep on top of it
    coverage = np.zeros_like(goal_density)
    ut = np.array(goal_density)

    # for keeping the runtime of each timestep
    time_arr = np.zeros(param.timesteps)

    agent.t = 0  # reset the agent's time
    # do absolute minimum inside the main loop
    for t in range(param.timesteps):
        dists, neighbor_ids, neighbor_coords = get_pcloud_neighbors(
            pcloud.pcd_tree,
            pcloud.vertices,
            np.copy(agent.x),
            agent.radius,
            param.nb_max_neighbors,
            param.nb_minimum_neighbors,
        )

        # Compute the coverage using RBF kernel
        kernel_vals = np.exp(-(1 / agent.radius) * dists**2)
        coverage[neighbor_ids] += kernel_vals

        neighbor_ids = neighbor_ids[: param.nb_minimum_neighbors]
        dists = dists[: param.nb_minimum_neighbors]
        neighbor_coords = pcloud.vertices[neighbor_ids, :]
        coverage_density = normalize_mat(coverage)
        source = np.maximum(goal_density - coverage_density, 0) ** 2
        source = normalize_mat(source)

        start_time = time.time()
        # Integrate the heat takes around ~ 2ms
        if param.method == "eigen":
            second_term = PhiT_M @ ut
            third_term = exp_vector * second_term
            ut = Phi @ third_term
        elif param.method == "exact":
            ut = A_invM @ source

        time_arr[t] = time.time() - start_time

        ut += param.source_strength * source
        (
            agent.x,
            gradient,
            _,
        ) = get_gradient(
            np.copy(agent.x),
            neighbor_coords,
            neighbor_ids,
            ut,
        )
        agent.update(gradient)

        coverage_arr[..., t] = coverage
        heat_arr[..., t] = np.copy(ut)
    return agent.x_arr, heat_arr, coverage_arr, time_arr

In [9]:
# Run and visualize a single experiment
# =================================================
x_arr, heat_arr, coverage_arr, time_arr = hedac(agent, param, pcloud)
print(f"Average Frequency: {1/np.mean(time_arr)} Hz ---")
val = 255-(normalize_mat(pcloud.u0)-normalize_mat(coverage_arr[:, 50]))
print(np.max(val))
# plots = visualize_point_cloud(
#     pcloud.vertices[val>50], colors=val[val>50], is_show_plot=False, point_size=3
# )
plots = visualize_point_cloud(
    pcloud.vertices, 
    colors=heat_arr[...,0], 
    # colors=heat_arr[...,-1], 
    is_show_plot=False, point_size=2
)
fig = visualize_trajectory(x_arr[:,:], plots, color="black")

fig.show()

# Compute and plot the coverage error
# ==========================
normalized_residual = compute_coverage_residual(heat_arr[..., 0], coverage_arr)

# plot the coverage error
fig = go.Figure()
# add traces for each error
fig.add_trace(
    go.Scatter(y=normalized_residual, mode="lines", name=f"Experiment")
)
fig.update_layout(
    xaxis_title="Timesteps",
    yaxis_title="Normalized Ergodic Metric",
    title="Normalized Coverage Residual",
)
fig.show()

Average Frequency: 1462.209470553989 Hz ---
255.0053032062016
